In [2]:
import pandas as pd
import os
import numpy as np
import pickle
import sklearn.preprocessing as preprocessing
import operator
import torch.nn as nn
import random
import torch
from torch.autograd import Variable

In [2]:
# root_path = 'data'
# files_path = os.listdir(root_path)

# all_list = []
# for file_path in files_path:
# # int(file_path.split('-')[0][1:])
# # int(file_path.split('-')[1][:8])
#     sample_pd = pd.read_csv(os.path.join(root_path, file_path))
#     sample_np = sample_pd.to_numpy()
#     if sample_np.shape == (121, 138):
#         # Record vehicle num
#         sample_np[1, -1] = file_path.split('-')[0][1:]
#         # Record date
#         sample_np[2, -1] = file_path.split('-')[1][:8]
#         all_list.append(sample_np)
# all_np = np.array(all_list)
# X = all_np[:, :, 2:-1].astype('float32')
# reshaped = X.reshape((-1, X.shape[-1]))
# scalerX = preprocessing.MaxAbsScaler().fit(reshaped)
# reshaped = scalerX.transform(reshaped)
# X = reshaped.reshape((X.shape[0], X.shape[1], X.shape[2]))
# all_np[:, :, 2:-1] = X
# pickle.dump(all_np, open('all_np.pkl', 'ab'))

In [3]:
with open('all_np.pkl', 'rb') as f: 
    all_np = pickle.load(f)
X = all_np[:, :, 2:-1].astype('float32')
Y = all_np[:, :3, -1]

In [5]:
all_np.shape

(1868, 121, 138)

In [11]:
uniq_code = [i[0] for i in Y]
uniq_code = set(uniq_code)
len(uniq_code)

78

In [12]:
uniq_code

{'P0000',
 'P000F',
 'P0045',
 'P007D',
 'P0087',
 'P0093',
 'P0097',
 'P0098',
 'P00CF',
 'P0113',
 'P0117',
 'P0171',
 'P0172',
 'P026A',
 'P0299',
 'P02CC',
 'P02CE',
 'P02D0',
 'P02D1',
 'P02D2',
 'P0300',
 'P0301',
 'P0303',
 'P0304',
 'P0401',
 'P0402',
 'P040B',
 'P040C',
 'P041B',
 'P041D',
 'P0420',
 'P0500',
 'P0507',
 'P052E',
 'P0567',
 'P0568',
 'P0571',
 'P0700',
 'P103C',
 'P1072',
 'P113A',
 'P1256',
 'P1380',
 'P13A1',
 'P13A2',
 'P13A3',
 'P13A4',
 'P1463',
 'P2002',
 'P2032',
 'P2033',
 'P20C9',
 'P20CB',
 'P20CD',
 'P20DE',
 'P2127',
 'P2147',
 'P2263',
 'P226B',
 'P2413',
 'P244B',
 'P244C',
 'P2454',
 'P2457',
 'P2459',
 'P2463',
 'P24A2',
 'P24AE',
 'P24B0',
 'P24C6',
 'P24DA',
 'P2563',
 'P2696',
 'P31B1',
 'U0001',
 'U0101',
 'U010E',
 'U02A3'}

In [67]:
# Statistic pcode
p_dict = {}
for i in range(X.shape[0]):
    if Y[i][0] not in p_dict.keys():
        p_dict[Y[i][0]] = 1
    else:
        p_dict[Y[i][0]] += 1

In [68]:
chosen_dict = {}
for i in p_dict.keys():
    if p_dict[i] >= 49:
        chosen_dict[i] = p_dict[i]

In [69]:
sorted_chosen_dict = chosen_dict
# Build dataframe
head_list = list(sorted_chosen_dict.keys())
head_list.append('count')
vehicle_pcode_df = pd.DataFrame(columns = head_list)

# Go over all the vehicles
v_dict = {}
for i in range(X.shape[0]):
    if Y[i][1] not in v_dict.keys():
        v_dict[Y[i][1]] = [Y[i][0]]
    else:
        v_dict[Y[i][1]].append(Y[i][0])
        
for vehicle in v_dict:
    new_dict = {}
    count = 0
    for pcode in sorted_chosen_dict:
        if pcode in v_dict[vehicle]:
            new_dict[pcode] = v_dict[vehicle].count(pcode)
            count += 1
        else:
            new_dict[pcode] = 0
    if count > 1:
        new_dict['count'] = count
        df = pd.DataFrame(new_dict, index =[vehicle])
        vehicle_pcode_df = vehicle_pcode_df.append(df)
        
test_vehicle_pcode_df = vehicle_pcode_df.loc[vehicle_pcode_df['count'] < 3]
training_vehicle_pcode_df = vehicle_pcode_df.loc[vehicle_pcode_df['count'] > 2]
training_vehicle_pcode_df = pd.concat([training_vehicle_pcode_df, test_vehicle_pcode_df.iloc[:int(test_vehicle_pcode_df.shape[0]/2)]])
test_vehicle_pcode_df = test_vehicle_pcode_df.iloc[int(test_vehicle_pcode_df.shape[0]/2):]
training_vehicle_pcode_df.to_csv('train.csv')
test_vehicle_pcode_df.to_csv('test.csv')

In [70]:
# Build training and test set
pcode_list = list(sorted_chosen_dict.keys())
training_vehicle_list = list(training_vehicle_pcode_df.index.values)
test_vehicle_list = list(test_vehicle_pcode_df.index.values)

training_dict = {}
test_dict = {}
for i in range(X.shape[0]):
    vehicle = Y[i, 1]
    pcode = Y[i, 0]
    date = int(Y[i, 2])
    if pcode in pcode_list:
        if vehicle in test_vehicle_list:
            if vehicle in test_dict.keys():
                if date in test_dict[vehicle].keys():
                    test_dict[vehicle][date] += [X[i:i+1, :, :], pcode]
                else:
                    test_dict[vehicle][date] = [X[i:i+1, :, :], pcode]
            else:
                test_dict[vehicle] = {date: [X[i:i+1, :, :], pcode]}

        if vehicle in training_vehicle_list:
            if vehicle in training_dict.keys():
                if pcode in training_dict[vehicle].keys():
                    training_dict[vehicle][pcode] = np.concatenate((training_dict[vehicle][pcode], X[i:i+1, :, :]))
                else:
                    training_dict[vehicle][pcode] = X[i:i+1, :, :]
            else:
                training_dict[vehicle] = {pcode: X[i:i+1, :, :]}
training_pcode = {}
for i in range(len(pcode_list)):
    pcode = pcode_list[i]
    l = []
    for j in range(len(training_vehicle_list)):
        training_vehicle_pcode_df[pcode][j]
        for k in range(training_vehicle_pcode_df[pcode][j]):
            l.append(training_vehicle_list[j])
    training_pcode[pcode_list[i]] = np.array(l)

In [125]:
data_length = 121
data_dim = 135
data_class = 10

x_context_sample_list = []
y_context_sample_list = []
x_target_sample_list = []
y_target_sample_list = []
# len(test_vehicle_list)
for vehicle in test_vehicle_list:
    sort_date_list = list(test_dict[vehicle].keys())
    if len(sort_date_list) > 2:
        sort_date_list.sort()


        # print(len(test_dict[vehicle][date]))
        for i in range(len(sort_date_list)-1, 0, -1):
            date = sort_date_list[i]
            for j in range(int(len(test_dict[vehicle][date])/2)):
                context_pointer = 0
                x_target_sample = test_dict[vehicle][date][2*j]
                y_target_sample = np.expand_dims(pcode_list.index(test_dict[vehicle][date][2*j+1]), axis=0)
                for k in range(i-1, -1, -1):
                    context_date = sort_date_list[k]
                    for l in range(int(len(test_dict[vehicle][context_date])/2)):
                        if context_pointer == 0:
                            x_context_sample = test_dict[vehicle][context_date][2*l]
                            y_context_sample = np.zeros([1, data_length, data_class])
                            y_context_sample[:, :, pcode_list.index(test_dict[vehicle][context_date][2*l+1])] += 1
                            context_pointer = 1
                        else:
                            x_context_sample = np.concatenate([x_context_sample, test_dict[vehicle][context_date][2*l]], axis=0)
                            sample = np.zeros([1, data_length, data_class])
                            sample[:, :, pcode_list.index(test_dict[vehicle][context_date][2*l+1])] += 1  
                            y_context_sample = np.concatenate([y_context_sample, sample], axis=0)
                x_context_sample_list.append(x_context_sample.copy())
                y_context_sample_list.append(y_context_sample.copy())
                x_target_sample_list.append(x_target_sample.copy())
                y_target_sample_list.append(y_target_sample.copy())


#     for date in sort_date_list:
#         if context_pointer == 0:
#             context_pointer = 1
#             if len(test_dict[vehicle][date]) == 2:
#                 x_context_sample = np.expand_dims(test_dict[vehicle][date][0], axis=0)
#     #             y_context_sample = np.expand_dims(test_dict[vehicle][date][1], axis=0)
#                 y_context_sample = np.zeros([1, data_length, data_class])
#                 y_context_sample[:, :, pcode_list.index(test_dict[vehicle][date][1])] += 1      
#             else:
#                 for i in range(int(len(test_dict[vehicle][date])/2)):
#                     sample_list = [k for k in range(int(len(test_dict[vehicle][date])/2))]
#                     sample_list.remove(i)
#                     context_set = 0
#                     x_target_sample = np.expand_dims(test_dict[vehicle][date][2*i], axis=0)
#                     y_target_sample = np.expand_dims(test_dict[vehicle][date][2*i+1], axis=0)
#                     # Build context set
#                     for j in sample_list:
#                         if context_set == 0:
#                             x_context_sample = np.expand_dims(test_dict[vehicle][date][2*j], axis=0)
#                             y_context_sample = np.expand_dims(test_dict[vehicle][date][2*j+1], axis=0)
#                             y_context_sample = np.zeros([1, data_length, data_class])
#                             y_context_sample[:, :, pcode_list.index(test_dict[vehicle][date][2*j+1])] += 1  
#                         else:
#                             x_context_sample = np.concatenate([x_context_sample, np.expand_dims(test_dict[vehicle][date][2*j], axis=0)], axis=0)
#                             sample = np.zeros([1, data_length, data_class])
#                             sample[:, :, pcode_list.index(test_dict[vehicle][date][2*j+1])] += 1  
#                             y_context_sample = np.concatenate([y_context_sample, sample], axis=0)
#                     x_context_sample_list.append(x_context_sample.copy())
#                     y_context_sample_list.append(y_context_sample.copy())
#                     x_target_sample_list.append(x_target_sample.copy())
#                     y_target_sample_list.append(y_target_sample.copy())
#         else:
#             if target_pointer == 0:
#                 pass


In [111]:
len(test_vehicle_list)

In [126]:
len(y_target_sample_list)

29

In [127]:
class NP_LSTM(nn.Module):
    def __init__(self):
        super(NP_LSTM, self).__init__()
        self.r_dim = 64
        self.z_dim = 64
        self.h_param = 500
        self.g_param = 500

        # LSTM parameters
        self.x_dim = 135
        self.y_dim = 10
        self.input_dim = 145
        self.output_dim = 10
        self.hidden_dim = 500
        self.num_layers = 2
        self.lstm_h = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        self.lstm_g = nn.LSTM(self.x_dim + self.z_dim, self.hidden_dim, self.num_layers)
        # Define output layer
        # self.linear = nn.Linear(self.hidden_dim, self.output_dim)
        self.g_l = nn.Linear(121 * self.g_param, self.output_dim)

        self.h_l = nn.Linear(self.h_param, self.r_dim)

        self.r_to_z_mean = nn.Linear(self.r_dim, self.z_dim)
        self.r_to_z_log_var = nn.Linear(self.r_dim, self.z_dim)

    def point_grad_to(self, target):
        '''
        Set .grad attribute of each parameter to be proportional
        to the difference between self and target
        '''
        for p, target_p in zip(self.parameters(), target.parameters()):
            if p.grad is None:
                # if self.is_cuda():
                p.grad = Variable(torch.zeros(p.size())).cuda()
                # else:
                #     p.grad = Variable(torch.zeros(p.size()))
            p.grad.data.zero_()  # not sure this is required
            p.grad.data.add_(p.data - target_p.data)

    def aggregate(self, rc):
        return torch.mean(torch.mean(rc, dim=0), dim=0)

    def h(self, input):
        h_out, self.hidden = self.lstm_h(input)
        h_out = self.h_l(h_out)
        return h_out

    def g(self, input):
        g_out, _ = self.lstm_g(input)
        g_out = torch.transpose(g_out, 0, 1)
        g_out = torch.reshape(g_out, (-1, 121 * self.hidden_dim))
        g_out = self.g_l(g_out)
        return g_out

    def xy_to_z(self, x, y):
        context = torch.cat([x, y], dim=2)
        rc = self.h(context)
        r = self.aggregate(rc)
        mu = self.r_to_z_mean(r)
        log_var = self.r_to_z_log_var(r)
        return mu, log_var

    def xz_to_y(self, x, z):
        std = torch.exp(0.5 * z[0])
        eps = torch.randn_like(std)
        z = eps.mul(std).add_(z[1])
        z = z.unsqueeze(0).expand(self.output_length, -1, -1)
#         print(z.shape)
#         print(x.shape)
        y = self.g(torch.cat([z, x], dim=2))
        return y

    def forward(self, x_context, y_context, x_target, x_all=None, y_all=None):
        # context = torch.cat([x_context, y_context.unsqueeze(2)], dim=2)
        # rC = self.h(context)
        # r = torch.mean(rC, dim=1)
        # mu = self.r_to_z_mean(r)
        # logVar = self.r_to_z_logVar(r)
        self.output_length = x_target.shape[0]
        z_context = self.xy_to_z(x_context, y_context)
        if x_all is not None:
            z_all = self.xy_to_z(x_all, y_all)
        else:
            z_all = None

        y_pred = self.xz_to_y(x_target, z_context)
        return y_pred, z_context, z_all

In [128]:
def kl_div_gaussians(mu_q, logvar_q, mu_p, logvar_p):
    var_p = torch.exp(logvar_p)
    kl_div = (torch.exp(logvar_q) + (mu_q - mu_p) ** 2) / var_p - 1.0 + logvar_p - logvar_q
    kl_div = 0.5 * kl_div.sum()
    return kl_div


def np_loss(y_pred, y_target, z_all, z_context):
    CELoss = nn.CrossEntropyLoss()(y_pred, y_target)
    KLD = kl_div_gaussians(z_all[0], z_all[1], z_context[0], z_context[1])
    return CELoss + KLD

In [129]:
# Start training
meta_lr = 1e-4
update_lr = 1e-4
update_step = 3
batch_size = 2
net = NP_LSTM().cuda()
steps = 10000
# Optimizers
meta_optimizer = torch.optim.Adam(net.parameters(), lr=meta_lr)
# CELoss = nn.CrossEntropyLoss()
task_order = [i for i in range(len(pcode_list))]
print('Start training')
best = 0
best_step = -1
# #######
for step in range(steps):
    random.shuffle(task_order)
    #######
    for task in task_order:


        # Choose data
        pcode = pcode_list[task]
        vehicle_list = training_pcode[pcode]
        vehicle_order = [i for i in range(len(vehicle_list))]
        random.shuffle(vehicle_order)
        batch_vehicle = vehicle_list[vehicle_order[:batch_size]]
        # Start building target set and context set for each vehicle
        # Start building target set and context set for each vehicle
        ######
        for vehicle in batch_vehicle:
        #     vehicle = batch_vehicle[0]
            # Build target
            # Build context
            if training_dict[vehicle][pcode].shape[0] > 1:
                pcode_chosen_one = random.randint(1,training_dict[vehicle][pcode].shape[0]) - 1
                pcode_chosen_list = [i for i in range(training_dict[vehicle][pcode].shape[0])]
                pcode_chosen_list.remove(pcode_chosen_one)
                x_target = training_dict[vehicle][pcode][pcode_chosen_one:pcode_chosen_one + 1, :, :]
                y_target = np.zeros([x_target.shape[0], data_length, data_class])
                y_target[:, :, task] += 1
                y_gt = np.array([task])
                x_context = training_dict[vehicle][pcode][pcode_chosen_list, :, :]
                if x_context.ndim < 3:
                    x_context = np.expand_dims(x_context, axis=0)
                y_context = np.zeros([x_context.shape[0], data_length, data_class])
                y_context[:, :, task] += 1
            else:
                x_target = training_dict[vehicle][pcode]
                y_target = np.zeros([x_target.shape[0], data_length, data_class])
                y_target[:, :, task] += 1
                y_gt = np.array([task])
                x_context = np.ones((0, data_length, data_dim))
                y_context = np.ones((0, data_length, data_class))
            other_pcode_list = list(training_dict[vehicle].keys())
            other_pcode_list.remove(pcode)
            for other_pcode in other_pcode_list:
                other_task = pcode_list.index(other_pcode)
                other_x_context = training_dict[vehicle][other_pcode]
                other_y_context = np.zeros([other_x_context.shape[0], data_length, data_class])
                other_y_context[:, :, other_task] += 1
                x_context = np.concatenate((x_context, other_x_context), axis=0)
                y_context = np.concatenate((y_context, other_y_context), axis=0)
#                 print(x_context.shape)
            x_batch = np.concatenate((x_context, x_target), axis=0)
            y_batch = np.concatenate((y_context, y_target), axis=0)
            x_context = torch.from_numpy(x_context).to(dtype=torch.float32).cuda().transpose(0, 1)
            y_context = torch.from_numpy(y_context).to(dtype=torch.float32).cuda().transpose(0, 1)
            x_target = torch.from_numpy(x_target).to(dtype=torch.float32).cuda().transpose(0, 1)
            y_gt = torch.from_numpy(y_gt).to(dtype=torch.float32).cuda().long()
            x_batch = torch.from_numpy(x_batch).to(dtype=torch.float32).cuda().transpose(0, 1)
            y_batch = torch.from_numpy(y_batch).to(dtype=torch.float32).cuda().transpose(0, 1)
            
            # Initialize network
            net_i = NP_LSTM().cuda()
            net_i.load_state_dict(net.state_dict())
            update_optimizer = torch.optim.Adam(net_i.parameters(), lr=update_lr)

            # Update process
            for j in range(update_step):
                y_pred, z_context, z_all = net_i(x_context, y_context, x_target, x_batch, y_batch)
                update_loss = np_loss(y_pred, y_gt, z_all, z_context)
                update_optimizer.zero_grad()
                update_loss.backward(retain_graph=True)
                update_optimizer.step()
            net.point_grad_to(net_i)
            meta_optimizer.step()
            
#             update_loss = np_loss(y_pred, y_gt, z_all, z_context)
#             y_pred_1 = y_pred.cpu().detach().numpy()
#             y_pred_1 = np.argmax(y_pred_1, axis=1)
# #             print('#####')
# #             print('target: ', y_target)
# #             print('pred: ', y_pred_1)
#             update_optimizer.zero_grad()
#             update_loss.backward(retain_graph=True)
#             update_optimizer.step()
# #         print('training loss: ', update_loss)

#         net.point_grad_to(net_i)
#         meta_optimizer.step()
        
# weight_name = 'model_%d.ckpt' % (step)
# torch.save(net.state_dict(), weight_name)
# weight_name = 'model_3.ckpt'
# net.load_state_dict(torch.load(weight_name))
    count = 0
    for i in range(len(x_context_sample_list)):
        x_context = torch.from_numpy(x_context_sample_list[i]).to(dtype=torch.float32).cuda().transpose(0, 1)
        y_context = torch.from_numpy(y_context_sample_list[i]).to(dtype=torch.float32).cuda().transpose(0, 1)
        x_target = torch.from_numpy(x_target_sample_list[i]).to(dtype=torch.float32).cuda().transpose(0, 1)
        y_target = y_target_sample_list[i]
        y_pred, _, _ = net(x_context, y_context, x_target)
        y_pred = y_pred.cpu().detach().numpy()
        y_pred = np.argmax(y_pred, axis=1)
        if y_pred == y_target:
            count += 1
    if count > best:
        best = count
        best_step = step
    print('step: ', step, ' || num: ', count, ' || best: ', best, ' || at: ', best_step)
            
            
        
#         x_context_sample_list = []
# y_context_sample_list = []
# x_target_sample_list = []
# y_target_sample_list = []
#     print(step)
        
#     print('epoch: %d || loss_1: %f' % (step, update_loss.data))
#     # # file = open('out' + '/log.txt', 'a')
#     # # file.write('task: %d || epoch: %d || loss_1: %f\n' % (i, step, meta_loss.data))
#     # # file.close()
#     if step % 100 == 0:
#         file = open('out/log.txt', 'a')
#         file.write('epoch: %d || loss_1: %f\n' % (step, update_loss.data))
#         file.close()
#     if step > 2000 and step % 200 == 0:
#         # Save weights
#         weight_name = 'out/model_%d.ckpt' % (step)
#         torch.save(net.state_dict(), weight_name)

Start training
step:  0  || num:  4  || best:  4  || at:  0
step:  1  || num:  4  || best:  4  || at:  0
step:  2  || num:  3  || best:  4  || at:  0
step:  3  || num:  9  || best:  9  || at:  3
step:  4  || num:  6  || best:  9  || at:  3
step:  5  || num:  5  || best:  9  || at:  3
step:  6  || num:  2  || best:  9  || at:  3
step:  7  || num:  5  || best:  9  || at:  3
step:  8  || num:  5  || best:  9  || at:  3
step:  9  || num:  5  || best:  9  || at:  3
step:  10  || num:  1  || best:  9  || at:  3
step:  11  || num:  8  || best:  9  || at:  3
step:  12  || num:  2  || best:  9  || at:  3
step:  13  || num:  8  || best:  9  || at:  3
step:  14  || num:  3  || best:  9  || at:  3
step:  15  || num:  3  || best:  9  || at:  3
step:  16  || num:  7  || best:  9  || at:  3
step:  17  || num:  6  || best:  9  || at:  3
step:  18  || num:  9  || best:  9  || at:  3
step:  19  || num:  7  || best:  9  || at:  3
step:  20  || num:  5  || best:  9  || at:  3
step:  21  || num:  8  || bes

step:  168  || num:  20  || best:  24  || at:  52
step:  169  || num:  17  || best:  24  || at:  52
step:  170  || num:  17  || best:  24  || at:  52
step:  171  || num:  14  || best:  24  || at:  52
step:  172  || num:  17  || best:  24  || at:  52
step:  173  || num:  16  || best:  24  || at:  52
step:  174  || num:  9  || best:  24  || at:  52
step:  175  || num:  13  || best:  24  || at:  52
step:  176  || num:  12  || best:  24  || at:  52
step:  177  || num:  14  || best:  24  || at:  52
step:  178  || num:  19  || best:  24  || at:  52
step:  179  || num:  13  || best:  24  || at:  52
step:  180  || num:  12  || best:  24  || at:  52
step:  181  || num:  17  || best:  24  || at:  52
step:  182  || num:  12  || best:  24  || at:  52
step:  183  || num:  17  || best:  24  || at:  52
step:  184  || num:  13  || best:  24  || at:  52
step:  185  || num:  12  || best:  24  || at:  52
step:  186  || num:  7  || best:  24  || at:  52
step:  187  || num:  11  || best:  24  || at:  52
st

KeyboardInterrupt: 

In [130]:
len(x_context_sample_list)

29

In [63]:
y_target.shape

(1,)

In [68]:
a = [1]
for i in range(len(a)-1, 0, -1):
    print(a[i])

In [14]:
len(x_context_sample_list)

29

In [30]:
pp = list(training_dict.keys())
training_dict[pp[9]].keys()

dict_keys(['P0402', 'P000F', 'P0420', 'P0093', 'P0087'])

In [117]:
len(x_context_sample_list)

26

In [ ]:
x_context, y_context, x_target, x_batch, y_batch

In [58]:
x_context.shape

torch.Size([121, 3, 135])

In [37]:
x_training = []
y_training = []
for vehicle in training_dict:
    v_dict = training_dict[vehicle]
    for pcode in v_dict:
        x_sample = training_dict[vehicle][pcode]
        for i in range(x_sample.shape[0]):
#             y_sample = np.zeros([data_class])
#             y_sample[pcode_list.index(pcode)] += 1
            x_training.append(x_sample[i].copy())
            y_training.append(pcode_list.index(pcode))
        
x_training = np.array(x_training)
y_training = np.array(y_training)

x_training = torch.from_numpy(x_training).to(dtype=torch.float32).cuda().transpose(0, 1)
y_training = torch.from_numpy(y_training).to(dtype=torch.float32).cuda().long()

In [52]:
x_test = []
y_test = []
for vehicle in test_dict:
    v_dict = test_dict[vehicle]
    for date in v_dict:
        sample = test_dict[vehicle][date]
        for i in range(len(sample) // 2):
            x_sample = sample[2 * i][0]
#             print(x_sample.shape)
            x_test.append(x_sample)
            y_test.append(pcode_list.index(sample[2 * i + 1]))
            
#         for i in range(x_sample.shape[0]):
#             y_sample = np.zeros([data_length, data_class])
#             y_sample[:, pcode_list.index(pcode)] += 1
#             x_test.append(x_sample[i].copy())
#             y_test.append(y_sample.copy())
        
x_test = np.array(x_test)
y_test = np.array(y_test)

x_test = torch.from_numpy(x_test).to(dtype=torch.float32).cuda().transpose(0, 1)


In [106]:
data_length = 121
data_dim = 135
data_class = 10

x_test = []
y_test = []
# len(test_vehicle_list)
for vehicle in test_vehicle_list:
    sort_date_list = list(test_dict[vehicle].keys())
    sort_date_list.sort()


    # print(len(test_dict[vehicle][date]))
    for i in range(len(sort_date_list)-1, 0, -1):
        date = sort_date_list[i]
        for j in range(len(test_dict[vehicle][date]) // 2):
            context_pointer = 0
            x_sample = test_dict[vehicle][date][2*j][0]
            y_sample = np.expand_dims(pcode_list.index(test_dict[vehicle][date][2*j+1]), axis=0)
            x_test.append(x_sample.copy())
            y_test.append(y_sample.copy())
x_test = np.array(x_test)
y_test = np.array(y_test)[:, 0]

x_test = torch.from_numpy(x_test).to(dtype=torch.float32).cuda().transpose(0, 1)

In [107]:
x_test.shape

torch.Size([121, 65, 135])

In [104]:
class LSTM(nn.Module):
    def __init__(self, hidden_dim, batch_size, task_num, num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = 135
        self.task_num = task_num
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        # Define output layer
        self.l1 = nn.Sequential(
            nn.Linear(121 * self.hidden_dim, task_num),
            )

    def forward(self, input):
        # shape of input  (seq_length, batch, input_dim)
        # shape of output (seq_length, batch, hidden_dim)
        y_pred, _ = self.lstm(input)
        y_pred = torch.transpose(y_pred, 0, 1)
        y_pred = torch.reshape(y_pred, (-1, 121 * self.hidden_dim))
        y_pred = self.l1(y_pred)
        return y_pred

#### y_training.shape

In [96]:
# Start training
task_num = 10
Step = 10000
batch_size = 5
update_lr = 1e-4
update_step = 5
input_dim = 135
hidden_dim = 500
best = 0
net = LSTM(hidden_dim, batch_size, task_num).cuda()
x_test_size = x_test.shape[1]

# Optimizers
optimizer = torch.optim.Adam(net.parameters(), lr=update_lr)
CELoss = nn.CrossEntropyLoss()

batch_list = [item for item in range(x_training.shape[0])]

print('Start training')
for step in range(Step):
    
    random.shuffle(batch_list)
    batch_content = batch_list[:batch_size]
    
    x_batch = x_training[:, batch_content, :]
    y_batch = y_training[batch_content]
    y_pred = net(x_batch)
    loss = CELoss(y_pred, y_batch)
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()

#     for i in range():
    y_pred_test = net(x_test)
    y_pred_test = y_pred_test.cpu().detach().numpy()
    y_pred_test = np.argmax(y_pred_test, axis=1)
    count = np.sum((y_test == y_pred_test) * 1)
        
#         if y_pred == y_target:
#             count += 1
    if count > best:
        best = count
        best_step = step
    print('step: ', step, ' || num: ', count, ' || best: ', best, ' || at: ', best_step)

Start training
step:  0  || num:  5  || best:  5  || at:  0
step:  1  || num:  0  || best:  5  || at:  0
step:  2  || num:  0  || best:  5  || at:  0
step:  3  || num:  0  || best:  5  || at:  0
step:  4  || num:  0  || best:  5  || at:  0
step:  5  || num:  0  || best:  5  || at:  0
step:  6  || num:  0  || best:  5  || at:  0
step:  7  || num:  0  || best:  5  || at:  0
step:  8  || num:  0  || best:  5  || at:  0
step:  9  || num:  0  || best:  5  || at:  0
step:  10  || num:  0  || best:  5  || at:  0
step:  11  || num:  0  || best:  5  || at:  0
step:  12  || num:  0  || best:  5  || at:  0
step:  13  || num:  0  || best:  5  || at:  0
step:  14  || num:  0  || best:  5  || at:  0
step:  15  || num:  0  || best:  5  || at:  0
step:  16  || num:  0  || best:  5  || at:  0
step:  17  || num:  0  || best:  5  || at:  0
step:  18  || num:  0  || best:  5  || at:  0
step:  19  || num:  0  || best:  5  || at:  0
step:  20  || num:  0  || best:  5  || at:  0
step:  21  || num:  0  || bes

step:  168  || num:  13  || best:  21  || at:  117
step:  169  || num:  12  || best:  21  || at:  117
step:  170  || num:  12  || best:  21  || at:  117
step:  171  || num:  12  || best:  21  || at:  117
step:  172  || num:  12  || best:  21  || at:  117
step:  173  || num:  14  || best:  21  || at:  117
step:  174  || num:  14  || best:  21  || at:  117
step:  175  || num:  14  || best:  21  || at:  117
step:  176  || num:  13  || best:  21  || at:  117
step:  177  || num:  15  || best:  21  || at:  117
step:  178  || num:  16  || best:  21  || at:  117
step:  179  || num:  16  || best:  21  || at:  117
step:  180  || num:  17  || best:  21  || at:  117
step:  181  || num:  21  || best:  21  || at:  117
step:  182  || num:  21  || best:  21  || at:  117
step:  183  || num:  18  || best:  21  || at:  117
step:  184  || num:  16  || best:  21  || at:  117
step:  185  || num:  16  || best:  21  || at:  117
step:  186  || num:  16  || best:  21  || at:  117
step:  187  || num:  15  || bes

step:  331  || num:  17  || best:  23  || at:  230
step:  332  || num:  17  || best:  23  || at:  230
step:  333  || num:  17  || best:  23  || at:  230
step:  334  || num:  17  || best:  23  || at:  230
step:  335  || num:  17  || best:  23  || at:  230
step:  336  || num:  17  || best:  23  || at:  230
step:  337  || num:  17  || best:  23  || at:  230
step:  338  || num:  16  || best:  23  || at:  230
step:  339  || num:  16  || best:  23  || at:  230
step:  340  || num:  16  || best:  23  || at:  230
step:  341  || num:  16  || best:  23  || at:  230
step:  342  || num:  15  || best:  23  || at:  230
step:  343  || num:  15  || best:  23  || at:  230
step:  344  || num:  16  || best:  23  || at:  230
step:  345  || num:  16  || best:  23  || at:  230
step:  346  || num:  16  || best:  23  || at:  230
step:  347  || num:  16  || best:  23  || at:  230
step:  348  || num:  15  || best:  23  || at:  230
step:  349  || num:  15  || best:  23  || at:  230
step:  350  || num:  13  || bes

step:  498  || num:  16  || best:  23  || at:  230
step:  499  || num:  15  || best:  23  || at:  230
step:  500  || num:  15  || best:  23  || at:  230
step:  501  || num:  15  || best:  23  || at:  230
step:  502  || num:  15  || best:  23  || at:  230
step:  503  || num:  16  || best:  23  || at:  230
step:  504  || num:  16  || best:  23  || at:  230
step:  505  || num:  16  || best:  23  || at:  230
step:  506  || num:  16  || best:  23  || at:  230
step:  507  || num:  18  || best:  23  || at:  230
step:  508  || num:  17  || best:  23  || at:  230
step:  509  || num:  13  || best:  23  || at:  230
step:  510  || num:  11  || best:  23  || at:  230
step:  511  || num:  7  || best:  23  || at:  230
step:  512  || num:  6  || best:  23  || at:  230
step:  513  || num:  10  || best:  23  || at:  230
step:  514  || num:  10  || best:  23  || at:  230
step:  515  || num:  13  || best:  23  || at:  230
step:  516  || num:  14  || best:  23  || at:  230
step:  517  || num:  15  || best:

step:  660  || num:  17  || best:  23  || at:  230
step:  661  || num:  17  || best:  23  || at:  230
step:  662  || num:  16  || best:  23  || at:  230
step:  663  || num:  16  || best:  23  || at:  230
step:  664  || num:  15  || best:  23  || at:  230
step:  665  || num:  15  || best:  23  || at:  230
step:  666  || num:  15  || best:  23  || at:  230
step:  667  || num:  15  || best:  23  || at:  230
step:  668  || num:  15  || best:  23  || at:  230
step:  669  || num:  15  || best:  23  || at:  230
step:  670  || num:  15  || best:  23  || at:  230
step:  671  || num:  15  || best:  23  || at:  230
step:  672  || num:  15  || best:  23  || at:  230
step:  673  || num:  15  || best:  23  || at:  230
step:  674  || num:  15  || best:  23  || at:  230
step:  675  || num:  16  || best:  23  || at:  230
step:  676  || num:  16  || best:  23  || at:  230
step:  677  || num:  15  || best:  23  || at:  230
step:  678  || num:  16  || best:  23  || at:  230
step:  679  || num:  15  || bes

step:  825  || num:  15  || best:  23  || at:  230
step:  826  || num:  15  || best:  23  || at:  230
step:  827  || num:  15  || best:  23  || at:  230
step:  828  || num:  14  || best:  23  || at:  230
step:  829  || num:  14  || best:  23  || at:  230
step:  830  || num:  14  || best:  23  || at:  230
step:  831  || num:  14  || best:  23  || at:  230
step:  832  || num:  15  || best:  23  || at:  230
step:  833  || num:  15  || best:  23  || at:  230
step:  834  || num:  15  || best:  23  || at:  230
step:  835  || num:  16  || best:  23  || at:  230
step:  836  || num:  17  || best:  23  || at:  230
step:  837  || num:  16  || best:  23  || at:  230
step:  838  || num:  15  || best:  23  || at:  230
step:  839  || num:  15  || best:  23  || at:  230
step:  840  || num:  13  || best:  23  || at:  230
step:  841  || num:  13  || best:  23  || at:  230
step:  842  || num:  13  || best:  23  || at:  230
step:  843  || num:  13  || best:  23  || at:  230
step:  844  || num:  13  || bes

step:  993  || num:  13  || best:  23  || at:  230
step:  994  || num:  15  || best:  23  || at:  230
step:  995  || num:  16  || best:  23  || at:  230
step:  996  || num:  16  || best:  23  || at:  230
step:  997  || num:  15  || best:  23  || at:  230
step:  998  || num:  15  || best:  23  || at:  230
step:  999  || num:  15  || best:  23  || at:  230
step:  1000  || num:  13  || best:  23  || at:  230
step:  1001  || num:  14  || best:  23  || at:  230
step:  1002  || num:  14  || best:  23  || at:  230
step:  1003  || num:  14  || best:  23  || at:  230
step:  1004  || num:  14  || best:  23  || at:  230
step:  1005  || num:  14  || best:  23  || at:  230
step:  1006  || num:  16  || best:  23  || at:  230
step:  1007  || num:  16  || best:  23  || at:  230
step:  1008  || num:  14  || best:  23  || at:  230
step:  1009  || num:  13  || best:  23  || at:  230
step:  1010  || num:  13  || best:  23  || at:  230
step:  1011  || num:  13  || best:  23  || at:  230
step:  1012  || num

step:  1158  || num:  12  || best:  23  || at:  230
step:  1159  || num:  12  || best:  23  || at:  230
step:  1160  || num:  12  || best:  23  || at:  230
step:  1161  || num:  12  || best:  23  || at:  230
step:  1162  || num:  12  || best:  23  || at:  230
step:  1163  || num:  12  || best:  23  || at:  230
step:  1164  || num:  12  || best:  23  || at:  230
step:  1165  || num:  12  || best:  23  || at:  230
step:  1166  || num:  12  || best:  23  || at:  230
step:  1167  || num:  12  || best:  23  || at:  230
step:  1168  || num:  12  || best:  23  || at:  230
step:  1169  || num:  12  || best:  23  || at:  230
step:  1170  || num:  12  || best:  23  || at:  230
step:  1171  || num:  12  || best:  23  || at:  230
step:  1172  || num:  12  || best:  23  || at:  230
step:  1173  || num:  12  || best:  23  || at:  230
step:  1174  || num:  12  || best:  23  || at:  230
step:  1175  || num:  12  || best:  23  || at:  230
step:  1176  || num:  12  || best:  23  || at:  230
step:  1177 

step:  1317  || num:  12  || best:  23  || at:  230
step:  1318  || num:  12  || best:  23  || at:  230
step:  1319  || num:  12  || best:  23  || at:  230
step:  1320  || num:  12  || best:  23  || at:  230
step:  1321  || num:  12  || best:  23  || at:  230
step:  1322  || num:  12  || best:  23  || at:  230
step:  1323  || num:  12  || best:  23  || at:  230
step:  1324  || num:  12  || best:  23  || at:  230
step:  1325  || num:  12  || best:  23  || at:  230
step:  1326  || num:  12  || best:  23  || at:  230
step:  1327  || num:  12  || best:  23  || at:  230
step:  1328  || num:  12  || best:  23  || at:  230
step:  1329  || num:  12  || best:  23  || at:  230
step:  1330  || num:  12  || best:  23  || at:  230
step:  1331  || num:  12  || best:  23  || at:  230
step:  1332  || num:  12  || best:  23  || at:  230
step:  1333  || num:  12  || best:  23  || at:  230
step:  1334  || num:  12  || best:  23  || at:  230
step:  1335  || num:  12  || best:  23  || at:  230
step:  1336 

KeyboardInterrupt: 

In [63]:
(y_test == y_pred_test) * 1

array([1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0])

In [64]:
y_pred_test.shape

(181,)

In [59]:
# Start training
meta_lr = 1e-4
update_lr = 1e-4
update_step = 5
batch_size = 2
net = NP_LSTM().cuda()
steps = 10000
# Optimizers
meta_optimizer = torch.optim.Adam(net.parameters(), lr=meta_lr)
# CELoss = nn.CrossEntropyLoss()
task_order = [i for i in range(len(pcode_list))]
print('Start training')
best = 0
best_step = -1
# #######
for step in range(steps):
    random.shuffle(task_order)
    #######
    for task in task_order:


        # Choose data
        pcode = pcode_list[task]
        vehicle_list = training_pcode[pcode]
        vehicle_order = [i for i in range(len(vehicle_list))]
        random.shuffle(vehicle_order)
        batch_vehicle = vehicle_list[vehicle_order[:batch_size]]
        # Start building target set and context set for each vehicle
        # Start building target set and context set for each vehicle
        ######
        for vehicle in batch_vehicle:
        #     vehicle = batch_vehicle[0]
            # Build target
            # Build context
            if training_dict[vehicle][pcode].shape[0] > 1:
                pcode_chosen_one = random.randint(1,training_dict[vehicle][pcode].shape[0]) - 1
                pcode_chosen_list = [i for i in range(training_dict[vehicle][pcode].shape[0])]
                pcode_chosen_list.remove(pcode_chosen_one)
                x_target = training_dict[vehicle][pcode][pcode_chosen_one:pcode_chosen_one + 1, :, :]
                y_target = np.zeros([x_target.shape[0], data_length, data_class])
                y_target[:, :, task] += 1
                y_gt = np.array([task])
                x_context = training_dict[vehicle][pcode][pcode_chosen_list, :, :]
                if x_context.ndim < 3:
                    x_context = np.expand_dims(x_context, axis=0)
                y_context = np.zeros([x_context.shape[0], data_length, data_class])
                y_context[:, :, task] += 1
            else:
                x_target = training_dict[vehicle][pcode]
                y_target = np.zeros([x_target.shape[0], data_length, data_class])
                y_target[:, :, task] += 1
                y_gt = np.array([task])
                x_context = np.ones((0, data_length, data_dim))
                y_context = np.ones((0, data_length, data_class))
            other_pcode_list = list(training_dict[vehicle].keys())
            other_pcode_list.remove(pcode)
            for other_pcode in other_pcode_list:
                other_task = pcode_list.index(other_pcode)
                other_x_context = training_dict[vehicle][other_pcode]
                other_y_context = np.zeros([other_x_context.shape[0], data_length, data_class])
                other_y_context[:, :, other_task] += 1
                x_context = np.concatenate((x_context, other_x_context), axis=0)
                y_context = np.concatenate((y_context, other_y_context), axis=0)
#                 print(x_context.shape)
            x_batch = np.concatenate((x_context, x_target), axis=0)
            y_batch = np.concatenate((y_context, y_target), axis=0)
            x_context = torch.from_numpy(x_context).to(dtype=torch.float32).cuda().transpose(0, 1)
            y_context = torch.from_numpy(y_context).to(dtype=torch.float32).cuda().transpose(0, 1)
            x_target = torch.from_numpy(x_target).to(dtype=torch.float32).cuda().transpose(0, 1)
            y_gt = torch.from_numpy(y_gt).to(dtype=torch.float32).cuda().long()
            x_batch = torch.from_numpy(x_batch).to(dtype=torch.float32).cuda().transpose(0, 1)
            y_batch = torch.from_numpy(y_batch).to(dtype=torch.float32).cuda().transpose(0, 1)
            
            # Initialize network
            net_i = NP_LSTM().cuda()
            net_i.load_state_dict(net.state_dict())
            update_optimizer = torch.optim.Adam(net_i.parameters(), lr=update_lr)

            # Update process
            for j in range(update_step):
                y_pred, z_context, z_all = net_i(x_context, y_context, x_target, x_batch, y_batch)
                update_loss = np_loss(y_pred, y_gt, z_all, z_context)
                update_optimizer.zero_grad()
                update_loss.backward(retain_graph=True)
                update_optimizer.step()
            net.point_grad_to(net_i)
            meta_optimizer.step()
            
#     count = 0
#     for i in range(len(x_context_sample_list)):
#         x_context = torch.from_numpy(x_context_sample_list[i]).to(dtype=torch.float32).cuda().transpose(0, 1)
#         y_context = torch.from_numpy(y_context_sample_list[i]).to(dtype=torch.float32).cuda().transpose(0, 1)
#         x_target = torch.from_numpy(x_target_sample_list[i]).to(dtype=torch.float32).cuda().transpose(0, 1)
#         y_target = y_target_sample_list[i]
#         y_pred, _, _ = net(x_context, y_context, x_target)
#         y_pred = y_pred.cpu().detach().numpy()
#         y_pred = np.argmax(y_pred, axis=1)
#         if y_pred == y_target:
#             count += 1
#     if count > best:
#         best = count
#         best_step = step
#     print('step: ', step, ' || num: ', count, ' || best: ', best, ' || at: ', best_step)


torch.Size([121, 1, 135])

In [27]:
y_training.shape

torch.Size([857, 10])

In [22]:
y_training[1] == y_training[0]

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False